In [ ]:
#순서가 중요 앞부분 features, 뒷부분 labels
def input_fn():
    return features, labels
    #return features, {'label1' : label1, 'label2' : label2}
    
def model_fn(features, labels, mode):
    return tf.estimator.EstimatorSpec(mode, train_op, loss)




In [33]:
import tensorflow as tf
import matplotlib.pyplot as plt

tf.logging.set_verbosity(tf.logging.INFO)
tf.reset_default_graph()

def parser(serialized_example):
    features = {
        'age': tf.FixedLenFeature([1], tf.int64),
        'img': tf.FixedLenFeature([61*49], tf.int64)
    }
    
    parsed_feature = tf.parse_single_example(serialized_example, features)
    age = parsed_feature['age']
    img = parsed_feature['img']
    return age, img

train_data_dir = './cnn_dataset/face_train.tfrecord'
test_data_dir = './cnn_dataset/face_test.tfrecord'
batch_size = 32

def get_input_fn(data_dir, batch_size):
    return lambda: input_fn(data_dir, batch_size)

def input_fn(data_dir, batch_size):
    dataset = tf.contrib.data.TFRecordDataset(data_dir).map(parser)
    dataset = train_dataset.shuffle(7777)
    dataset = train_dataset.batch(batch_size)
    
    itr = dataset.make_one_shot_iterator()
    
    age, img = itr.get_next()
    
    img = tf.reshape(img, [-1, 61, 49, 1])
    img = tf.cast(img, tf.float32)

    age = tf.reshape(age, [-1])
    age = tf.one_hot(age, depth=3, axis=-1, dtype=tf.float32)
    
    return img, age


def cnn_model(features, labels, mode):
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PRED = mode == tf.estimator.ModeKeys.PREDICT
    
    conv1 = tf.layers.conv2d(features, 
                             filters=16, 
                             kernel_size=4, 
                             padding='same', 
                             activation=tf.nn.relu,  
                             name='conv1')
    print(conv1.shape)
    conv2 = tf.layers.conv2d(conv1,
                            filters=32,
                            kernel_size=3,
                            padding='same',
                            activation=tf.nn.relu,
                            name='conv2')
    print(conv2.shape)
    pool2 = tf.layers.max_pooling2d(conv2, pool_size=2, strides=2)
    print(pool2.shape)

    conv3 = tf.layers.conv2d(pool2,
                            filters=64,
                            kernel_size=3,
                            padding='same',
                            activation=tf.nn.relu,
                            name='conv3')
    print(conv3.shape)
    pool3 = tf.layers.max_pooling2d(conv3, pool_size=2, strides=2)
    print(pool3.shape)

    flat = tf.layers.flatten(pool3)
    print(flat.shape)

    fc1 = tf.layers.dense(flat, 
                          units=1000, 
                          activation=tf.nn.relu, 
                          name='fc1')
    
    drop1 = tf.layers.dropout(fc1, 0.3, training=TRAIN)

    print(fc1.shape)
    fc2 = tf.layers.dense(drop1,
                         units=200,
                         activation=tf.nn.relu,
                         name='fc2')
    print(fc2.shape)
    
    drop2 = tf.layers.dropout(fc2, 0.3, training=TRAIN)

    out = tf.layers.dense(drop2,
                         units=3,
                         name='out')
    print(out.shape)
    global_step = tf.train.get_global_step()
    
    estimator_spec = None
    if TRAIN:
        loss = tf.losses.softmax_cross_entropy(age, out)
        train_op = tf.train.GradientDescentOptimizer(1e-5).minimize(loss, global_step)
        estimator_spec = tf.estimator.EstimatorSpec(mode=mode,
                                     loss=loss,
                                     train_op=train_op)
    elif EVAL:
        loss = tf.losses.softmax_cross_entropy(age, out) 
        pred = tf.nn.softmax(out)
        accuracy = tf.metrics.accuracy(tf.argmax(labels, 1), tf.argmax(pred, 1))
        eval_metric_ops = {'acc' : accuracy}
        estimator_spec = tf.estimator.EstimatorSpec(mode=mode,
                                     loss=loss,
                                     eval_metric_ops=eval_metric_ops)
    elif PRED:
        prob = tf.nn.softmax(out)
        age = tf.argmax(prob, axis=1)
        pred = {'img' : img, 'prob' : prob, 'age' : age}
        estimator_spec = tf.estimator.EstimatorSpec(mode=mode,
                                                   predictions=pred)
        
        
        
    else:
        raise Exception('estimator spec is None')
        
    return estimator_spec


cnn_estimator = tf.estimator.Estimator(cnn_model, model_dir='./cnn_models_B/')
cnn_estimator.train(get_input_fn(train_data_dir, batch_size))
cnn_estimator.evaluate(get_input_fn(test_data_dir, batch_size))
_preds = cnn_estimator.predict(get_input_fn(test_data_dir, batch_size))

for p in _preds:
    print(p['prob'], p['age'])
    img = np.mean(p['img'], axis=-1)
    
    plt.imshow(img)
    plt.show()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_service': None, '_num_ps_replicas': 0, '_log_step_count_steps': 100, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_model_dir': './cnn_models_B/', '_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1179c1f60>, '_save_checkpoints_secs': 600, '_master': '', '_task_id': 0, '_session_config': None, '_num_worker_replicas': 1, '_save_checkpoints_steps': None}
(?, 61, 49, 16)
(?, 61, 49, 32)
(?, 30, 24, 32)
(?, 30, 24, 64)
(?, 15, 12, 64)
(?, 11520)
(?, 1000)
(?, 200)
(?, 3)


UnboundLocalError: local variable 'age' referenced before assignment

In [ ]:
#     pred = tf.nn.softmax(test_out)
#     accuracy = tf.metrics.accuracy(tf.argmax(age, 1), tf.argmax(pred, 1))

# tf.summary.scalar('loss', loss)
# for v in tf.trainable_variables():
#     tf.summary.histogram('var_{}'.format(v.name), v)

# merged = tf.summary.merge_all()

In [15]:
import tensorflow as tf
import matplotlib.pyplot as plt

tf.logging.set_verbosity(tf.logging.INFO)
tf.reset_default_graph()

def parser(serialized_example):
    features = {
        'age': tf.FixedLenFeature([1], tf.int64),
        'img': tf.FixedLenFeature([61*49], tf.int64)
    }
    
    parsed_feature = tf.parse_single_example(serialized_example, features)
    age = parsed_feature['age']
    img = parsed_feature['img']
    return age, img

train_data_dir = './cnn_dataset/face_train.tfrecord'
test_data_dir = './cnn_dataset/face_test.tfrecord'
batch_size = 32

def get_input_fn(data_dir, batch_size):
    return lambda: input_fn(data_dir, batch_size)

def input_fn(data_dir, batch_size):
    dataset = tf.contrib.data.TFRecordDataset(data_dir).map(parser)
    dataset = dataset.shuffle(7777)
    dataset = dataset.batch(batch_size)
    
    itr = dataset.make_one_shot_iterator()
    
    age, img = itr.get_next()
    
    img = tf.reshape(img, [-1, 61, 49, 1])
    img = tf.cast(img, tf.float32)

    age = tf.reshape(age, [-1])
    age = tf.one_hot(age, depth=3, axis=-1, dtype=tf.float32)

    return img, age



def cnn_model(features, labels, mode):
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PRED = mode == tf.estimator.ModeKeys.PREDICT
    
    conv1 = tf.layers.conv2d(features, 
                             filters=16,
                             kernel_size=4, 
                             padding='same', 
                             activation=tf.nn.relu, 
                             name='conv1')
    print(conv1.shape)
    conv2 = tf.layers.conv2d(conv1,
                            filters=32,
                            kernel_size=3,
                            padding='same',
                            activation=tf.nn.relu,
                            name='conv2')
    print(conv2.shape)
    pool2 = tf.layers.max_pooling2d(conv2, pool_size=2, strides=2)
    print(pool2.shape)

    conv3 = tf.layers.conv2d(pool2,
                            filters=64,
                            kernel_size=3,
                            padding='same',
                            activation=tf.nn.relu,
                            name='conv3')
    print(conv3.shape)
    pool3 = tf.layers.max_pooling2d(conv3, pool_size=2, strides=2)
    print(pool3.shape)

    flat = tf.layers.flatten(pool3)
    print(flat.shape)

    fc1 = tf.layers.dense(flat, 
                          units=1000, 
                          activation=tf.nn.relu, 
                          name='fc1')
    
    drop1 = tf.layers.dropout(fc1, 0.3, training=TRAIN)

    print(fc1.shape)
    fc2 = tf.layers.dense(drop1,
                         units=200,
                         activation=tf.nn.relu,
                         name='fc2')
    print(fc2.shape)
    
    drop2 = tf.layers.dropout(fc2, 0.3, training=TRAIN)

    out = tf.layers.dense(drop2,
                         units=3,
                         name='out')
    print(out.shape)
    
    global_step = tf.train.get_global_step()
    
    estimator_spec = None
    if TRAIN:
        loss = tf.losses.softmax_cross_entropy(labels, out)
        train_op = tf.train.GradientDescentOptimizer(1e-5).minimize(loss, global_step)
        estimator_spec = tf.estimator.EstimatorSpec(mode=mode,
                                     loss=loss,
                                     train_op=train_op)
    elif EVAL:
        loss = tf.losses.softmax_cross_entropy(labels, out)
        pred = tf.nn.softmax(out)
        accuracy = tf.metrics.accuracy(tf.argmax(labels, 1), tf.argmax(pred, 1))
        eval_metric_ops = {'acc': accuracy}
        estimator_spec = tf.estimator.EstimatorSpec(mode=mode,
                                     loss=loss,
                                     eval_metric_ops=eval_metric_ops)
    else:
        raise Exception('Estimator spec is None')

    return estimator_spec

cnn_estimator = tf.estimator.Estimator(cnn_model, model_dir='./cnn_models_B/')
# cnn_estimator.train(get_input_fn(train_data_dir, batch_size))
cnn_estimator.evaluate(get_input_fn(test_data_dir, batch_size))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_service': None, '_num_ps_replicas': 0, '_log_step_count_steps': 100, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_model_dir': '/var/folders/9f/r85pl3x54ysbtzyrt_jv00y40000gn/T/tmphxsjswu2', '_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x117acacc0>, '_save_checkpoints_secs': 600, '_master': '', '_task_id': 0, '_session_config': None, '_num_worker_replicas': 1, '_save_checkpoints_steps': None}


ValueError: Could not find trained model in model_dir: /var/folders/9f/r85pl3x54ysbtzyrt_jv00y40000gn/T/tmphxsjswu2.

## map

In [18]:
str_list = '1 2 3 4 5 6 7'.split(' ')
print(str_list[0])

int_list = []
for i in str_list:
    int_list.append(int(i))
print(int_list)


1
[1, 2, 3, 4, 5, 6, 7]


In [22]:
int_list = list(map(int, str_list))
print(int_list)

[1, 2, 3, 4, 5, 6, 7]


In [25]:
def add(x):
    return x + 10

print(add(5))

list(map(add, [1, 2, 3, 4, 5, 6]))

15


[11, 12, 13, 14, 15, 16]

In [26]:
def mul(x):
    return x * 4

list(map(mul, [1, 2, 3, 4, 5, 6]))

[4, 8, 12, 16, 20, 24]

In [28]:
power = lambda x: x*x

list(map(power, [1, 2, 3, 4, 5, 6]))
list(map(lambda x: x*x, [1, 2, 3, 4, 5, 6]))

[1, 4, 9, 16, 25, 36]

In [31]:
def equation(w, b):
    return lambda x: x*w + b

print(equation(3, 5)(1))

linear = equation(3, 5)
print(linear)
result = linear(2)
print(result)

8
<function equation.<locals>.<lambda> at 0x117c5e2f0>
11
